## TENSORS
- https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html

In [ ]:
import numpy as np
import torch

### Initializing a Tensor

In [ ]:
# directry from data
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)

In [ ]:
x_data

In [ ]:
# np
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

In [ ]:
x_np

In [ ]:
# all one
x_ones = torch.ones_like(x_data)  # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

# random number
x_rand = torch.rand_like(x_data, dtype=torch.float)  # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

In [ ]:
# With random or constant values
shape = (
    2,
    3,
)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

### Attributes of a Tensor

In [ ]:
tensor = torch.rand(3, 4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(
    f"Device tensor is stored on: {tensor.device}"
)  # Is the torch.device where this Tensor is.

### Operations on Tensors

In [ ]:
"""
We move our tensor to the GPU if available.
By default, tensors are created on the CPU.
We need to explicitly move tensors to the GPU using .to method.
"""
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

In [ ]:
# tensor形式であってもnumpyの様な配列操作、統計値計算、fillが可能（省略）

## DATASETS & DATALOADERS
- https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

### Loading a Dataset

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",  # save path
    train=True,  # train or test
    download=True,  # if none download from internet
    transform=ToTensor(),  # convert data
)

test_data = datasets.FashionMNIST(
    root="data",  # save path
    train=False,  # train or test
    download=True,  # id none download from internet
    transform=ToTensor(),  # convert data
)

### Iterating and Visualizing the Dataset

In [ ]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

### Creating a Custom Dataset for your files

In [ ]:
import os

import pandas as pd
from torchvision.io import read_image


class CustomImageDataset(Dataset):
    # dataset objectをインスタンス化するときに実行. 各パラメータの初期化
    def __init__(
        self, annotations_file, img_dir, transform=None, target_transform=None
    ):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    # get number of samples
    def __len__(self):
        return len(self.img_labels)

    # 与えられたindexのデータを取得する
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

### Preparing your data for training with DataLoaders

In [ ]:
from torch.utils.data import DataLoader

# シャッフル、batch化をiterableによろしくやってくれる
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

### Iterate through the DataLoader

In [ ]:
# Display image and label.
# dataloaderにロードしているので、反復処理ができる？　（毎回読む必要がないことを言いたい気がする）
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [ ]:
train_features, train_labels = next(iter(train_dataloader))  # nextなので次の画像が呼び出される
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

## TRANSFORMS
- https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html

In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import Lambda, ToTensor

In [ ]:
"""
The FashionMNIST features are in PIL Image format, and the labels are integers.
For training, we need the features as normalized tensors, and the labels as one-hot encoded tensors.
To make these transformations, we use ToTensor and Lambda.
"""
ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),  # converts a PIL image or NumPy ndarray into a FloatTensor
    target_transform=Lambda(
        lambda y: torch.zeros(10, dtype=torch.float).scatter_(
            0, torch.tensor(y), value=1
        )
    ),  # one hot encod using lambdafunction
)

## BUILD THE NEURAL NETWORK
- https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [ ]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get Device for Training

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

### Define the Class
- `nn.Module`のサブクラスとして定義をする
- `__init__`でNNを初期化する
- `forward`で伝搬部分の処理を記述する
  - kerasは良き感じにここをやってくれていた
  - 各レイヤー、ユニット数を決め、伝搬部も記述すると考えると自然ではある

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
# 明示的にCPU、GPUどちらで処理をするのか記述
# 先に実行計画（モデル）を渡し、入力が来たらそれを実行するイメージ？
model = NeuralNetwork().to(device)
print(model)

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)  # get 10 logits
pred_probab = nn.Softmax(dim=1)(logits)  # softmax
y_pred = pred_probab.argmax(1)  # get proba
print(f"Predicted class: {y_pred}")

### Model Layers

In [ ]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

In [ ]:
# flatten
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

In [ ]:
# linear
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

In [ ]:
# relu
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

In [ ]:
# sequential
seq_modules = nn.Sequential(flatten, layer1, nn.ReLU(), nn.Linear(20, 10))
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [ ]:
# softmax
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model Parameters


In [ ]:
# you can preview each parameter
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

## Automatic Differentiation with torch.autograd
- https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html#

In [ ]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)  # learning param
b = torch.randn(3, requires_grad=True)  # learning param
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

<img src='https://pytorch.org/tutorials/_images/comp-graph.png'>

In [ ]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

### Computing Gradients

In [ ]:
loss.backward()  # calc derivatives
print(w)
print(w.grad)
print(b)
print(b.grad)

### Disabling Gradient Tracking

In [ ]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

# 順方向の計算結果のみが必要な場合
with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

# another way
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

## OPTIMIZING MODEL PARAMETERS
- https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import Lambda, ToTensor

training_data = datasets.FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data", train=False, download=True, transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

### Hyperparameters

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### Optimization Loop
- epoch
  - The Train Loop - iterate over the training dataset and try to converge to optimal parameters.
  - The Validation/Test Loop - iterate over the test dataset to check if model performance is improving.

### Loss Function

In [ ]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimizer

- `optimizer.zero_grad()`: 勾配の初期化
- `loss.backward()`: bp
- `optimizer.step()`: update params

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Full Implementation

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")